# Strand data build: SP500 data

## Build functions

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import requests
import sys
from tiingo import TiingoClient

config = {}
config['session'] = True
config['api_key'] =  "my_tiingo_api_key"
client = TiingoClient(config)

In [2]:
# Establish dict for stock ticker to cik conversion
ticker_cik = pd.read_csv("https://www.sec.gov/include/ticker.txt", sep = "\t", header = None)
cik_dict = ticker_cik.set_index(0).to_dict()[1]

# Function to establish the filing index pages
def get_index_url(symbol, date):
    # Get cik number for given symbol.
    cik = cik_dict[symbol]
    
    # Initialize filing_date arbitrarily early date for use later.
    filing_date = datetime.strptime('2000-01-01', '%Y-%m-%d').date()
    
    # Our goal is to find most recently filed corporate info - 
    # could be in a 10-Q or 10-K so we check both.
    for filing in ['10-Q', '10-K']:
        
        # Find SEC filings page for current symbol, ticker, and date using wildcards.
        base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}&owner=include&count=100"
        filing_page_resp = requests.get(base_url.format(cik, filing, date))
        filing_page_str = filing_page_resp.text

        # BS allows us to parse the HTML. Establish that HTML 'soup',
        # find table of filings by class, and establish table rows by tag.
        soup = BeautifulSoup(filing_page_str, 'html.parser')
        table = soup.find('table', class_='tableFile2')
        rows = table.find_all('tr')
        
        # Iterate over table rows, getting cells by tag. We break
        # once we make sure that we find the right filing (and we
        # check to ensure the right filing mostly because we could 
        # get mixed up by '10-K/A' amendments which appear in '10-K' 
        # search but aren't useful).
        for row in rows:
            cells = row.find_all('td')
            # If this is True, we've reached table observations and
            # found a relevant filing.
            # We want to only use information we could have known
            # PRIOR to the trading day, so I'll move past filings
            # that were submitted on the same day as the input date.
            if len(cells) > 3 and cells[0].text == filing and cells[3].text != date:
                # We'll update link and filing_date IF date (cells[3]) is more
                # recent than saved filing_date (to find most recent
                # filing between 10-Q and 10-K pages).
                check_date = datetime.strptime(cells[3].text, '%Y-%m-%d').date()
                if check_date > filing_date:
                    index_url = 'https://www.sec.gov' + cells[1].a['href']
                    filing_date = check_date
                    filing_type = filing
                break
                    
    return(index_url, filing_date, filing_type)
           

# Function to establish relevant xbrl links.
# All we need to do is pull index page for most recent company filing
# and find the 'XBRL INSTANCE DOCUMENT'.
def get_xbrl_url(index_url):    

    # As in get_index_url() function, establish page text.
    index_resp = requests.get(index_url)
    index_str = index_resp.text
    
    # As in get_index_url() function, get BS 'soup', find our table
    # of interest (in this case, doing so by table's summary), and
    # establish rows.
    soup = BeautifulSoup(index_str, "html.parser")
    table = soup.find('table', summary='Data Files')
    rows = table.find_all('tr')
    
    # Iterate over table rows looking for XBRL instance doc, saving link
    # once we find it.
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 3 and 'XBRL INSTANCE DOCUMENT' in cells[1].text:
            xbrl_link = 'https://www.sec.gov' + cells[2].a['href']
                
    return(xbrl_link)


# Get most recent values for our desired tags within the xbrl file.
def get_tag_values(xbrl_link):

    # As above, get XBRL text.
    xbrl_resp = requests.get(xbrl_link)
    xbrl_str = xbrl_resp.text
    
    # This time, our 'soup' is 'lxml' format instead of 'html'
    soup = BeautifulSoup(xbrl_str, 'lxml')
    
    # At some stage, companies swapped from manual submission
    # of their 'XBRL INSTANCE DOCUMENT' to iXBRL submission, resulting
    # in 'EXTRACTED XBRL INSTANCE DOCUMENT's. The only relevant change 
    # this causes is that a tag's 'xbrli:context' became 'context' and
    # the 'xbrli:enddate' became 'enddate'. We'll set these conditional
    # on which version the file is using, as identified by whether we
    # find an 'xbrli:context' tag in the file.
    xbrli_string = ''
    if soup.find('xbrli:context'):
        xbrli_string = 'xbrli:'
    
    context_tag = xbrli_string + 'context'
    enddate_tag = xbrli_string + 'enddate'
    instant_tag = xbrli_string + 'instant'
    
    # Here, we'll pull the filing's reference period end
    # (only for the sake of additional detail)
    ref_period = soup.find('dei:documentfiscalperiodfocus')
    date_id = ref_period.get('contextref')
    period_context = soup.find(context_tag, {'id':date_id})
    period_end = period_context.find(enddate_tag).text
    
    
    # Now we get into pulling our equity values. Some key notes:
    
    # 1. We are only interested in these tags if they are true values, 
    # not 'dimensions' of those values. We check this by ensuring
    # the tag's context does NOT contain an 'explicitmember' field.
    
    # 2. We only want to keep the most recent value (found in the tag's 
    # context), because filings can list values for past filings as well.
    
    # 3. We are interested in equity not including minority interest.
    # 'stockholdersequity' =
    # 'stockholdersequityincludingportionattributabletononcontrollinginterest'
    # less 'minorityinterest'. If we are missing 'stockholdersequity', we can
    # therefore calculate it with the other two figrues. In some cases, companies will
    # list only 'stockholdersequityincludingportionattributabletononcontrollinginterest'
    # without the other two figures. In these cases, it appears that the company does
    # not have any 'minorityinterest' and we can use this value as our equity figure.
    
    # For more details, see:
    # - https://xbrl.us/data-rule/dqc_0004pr/
    # - https://xbrl.us/guidance/specific-non-controlling-interest-elements/
    
    # Here, we'll get the filing's most recent values for each tag and save to dict.
    equity_dict = {}
    for equity_tag in ["us-gaap:stockholdersequity", 
                       "us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest", 
                       "us-gaap:minorityinterest"]:
        tag_dict = {"date": None,
                   "value": None}
        
        # Find tags in document.
        tag_list = soup.find_all(equity_tag)
        
        # Initialize for use in FOR loop.
        equity_date = datetime.strptime('2000-01-01', '%Y-%m-%d').date()
        
        for t in tag_list:
            # Get the tag's context for additional detail.
            ref_id = t.get('contextref')
            context = soup.find(context_tag, {'id':ref_id})
            
            # Pull the value's date reference from its context.
            equity_date_obs = datetime.strptime(context.find(instant_tag).text, '%Y-%m-%d').date()
            
            # If the value is more recent than saved value AND NOT a 'dimension',
            # we update the reference date and our saved value.
            if equity_date_obs >= equity_date and not context.find('xbrldi:explicitmember'):
                equity_date = equity_date_obs
                tag_dict = {"date": equity_date,
                           "value": int(float(t.text))}
                
        # Coming out of the above loop, we now have the 'value' of
        # the most recent non-'dimension' tag and its 'date'. We save 
        # these, and the tag name for reference, to the equity_dict.
        equity_dict.update({equity_tag: tag_dict})
        
    # To go alongside the logic in point (3.), we'll make sure to use the most recent
    # equity figure if the figures' dates including and excluding minority interest differ.
    # This did not occur in any of my trials but it seems like an edge case worth covering
    # since I've found no explicit rules against it.
    most_recent = True
    if equity_dict["us-gaap:stockholdersequity"]['value'] is not None and equity_dict["us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest"]['value'] is not None:
        if equity_dict["us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest"]['date'] >= equity_dict["us-gaap:stockholdersequity"]['date']:
            most_recent = False
    
    # So, with our equity_dict, we can save a single equity value
    # using the logic described in point (3.) above.
    if equity_dict["us-gaap:stockholdersequity"]['value'] is not None and most_recent:
        equity = equity_dict["us-gaap:stockholdersequity"]['value']
        equity_date = equity_dict["us-gaap:stockholdersequity"]['date']
    else:
        if equity_dict["us-gaap:minorityinterest"]['value'] is not None:
            equity = equity_dict["us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest"]['value'] - equity_dict["us-gaap:minorityinterest"]['value']
        else:
            equity = equity_dict["us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest"]['value']
        equity_date = equity_dict["us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest"]['date']
    
    # Now we'll look to establish the shares outstanding figure.
    # This is a bit simpler. According to the XBRL rules, each
    # class of stock should have one and only one value in the 
    # document tagged 'entitycommonstocksharesoutstanding'. We 
    # can run through these tags and if we find a 'dimension'-less
    # value we save and stop (this is the total of shares outstanding),
    # and if we don't then we sum the 'dimension'-al values. We only
    # do not blindly sum all tags to handle the case where a 
    # 'dimension'-less sum AND class-wise aspects are included 
    # (I don't think this should happen, but again I've found no
    # explicit rule against it).
    
    # For additional detail, see:
    # - https://www.sec.gov/structureddata/edgarvalidationerrors
    
    for shares_tag in ["dei:entitycommonstocksharesoutstanding"]:
        # Establish tag list
        tag_list = soup.find_all(shares_tag)
        
        shares = 0
        
        # Iterate over tag list
        for t in tag_list:
            ref_id = t.get('contextref')
            context = soup.find(context_tag, {'id':ref_id})
            
            # If we find a 'dimension'-less value, this contains the total
            # shares so we can save and break
            if not context.find('xbrldi:explicitmember'):
                shares = int(float(t.text))
                shares_date = datetime.strptime(context.find(instant_tag).text, '%Y-%m-%d').date()
                break
                
            # If we only find 'dimension'-al values, we'll sum them.
            else:
                shares += int(float(t.text))
                shares_date = datetime.strptime(context.find(instant_tag).text, '%Y-%m-%d').date()
        
    # Of note - we save the date of the shares observation for
    # later market cap calculations. And we save equity_date
    # for completeness - Though this seems unnecessary because
    # it appears to always be the same as period_end.
    return(period_end, equity, equity_date, shares, shares_date)


# Iterate over symbols and end dates to bring together
# our sample_inputs (though we'll need to update with
# prices to get market cap).
def get_sample_inputs(symbols, dates):
    
    sample_inputs = pd.DataFrame()
    
    for symbol in symbols:
        for date in dates:
            # Get index url for the filing we want w/ filing information
            # try:
            index_url, filing_date, filing_type = get_index_url(symbol, date)
            # except UnboundLocalError:
            # pass
            
            # Get xbrl url from index page 'index_url'
            xbrl_url = get_xbrl_url(index_url)
            # Get tag values from the XBRL instance file 'xbrl_url'
            period_end, equity, equity_date, shares, shares_date = get_tag_values(xbrl_url)
            
            # Get share price on most recent trading day prior to shares count observation.
            # Using Tiingo API (requires session config as described at:
            # https://tiingo-python.readthedocs.io/en/latest/readme.html#further-docs).
            # Tiingo will throw an error if the date we try has no pricing data. So
            # when the function fails we go back a day and try again.
#             shares_date_check = shares_date
#             while True:
#                 try:
#                     share_price = client.get_dataframe(symbol, 
#                                                        metric_name = 'close',
#                                                        startDate = shares_date_check,
#                                                        endDate = shares_date_check)[0]
#                     break
#                 except:
#                     shares_date_check = shares_date_check - timedelta(days=1)
                
            
#             # Calculate market cap on the date of the shares observation. 
#             market_cap = shares * share_price
            
            # Bring values together into dataframe rows and append to full frame.
            row_values = [symbol, cik_dict[symbol], date, filing_date, period_end, filing_type, equity, equity_date, shares, shares_date, index_url, xbrl_url, datetime.now()]
            names = ['symbol', 'cik', 'end_date', 'filing_date', 'period_end', 'filing_type', 'total_equity', 'equity_date', 'shares_outstanding', 'shares_date', 'index_url', 'xbrl_url', 'download_datetime']
            row = pd.DataFrame([row_values], columns=names)
            sample_inputs = sample_inputs.append(row)
            
    return(sample_inputs)

In [3]:
def get_sample_pricing(symbols, date_range):
    
    # Establish dates for Tiingo use.
    start = date_range[0]
    end = date_range[1]
    
    # Establish empty input_prices for reference when building the
    # input_prices frame.
    sample_pricing = pd.DataFrame(columns = ['symbol', 'date'])
    
    # Using Tiingo, we can either get all metrics for one symbol at
    # a time or get one metric for many symbols at once. It feels
    # more sensible to loop through our 4 metrics of interest than
    # to loop through each symbol and select out those metrics.
    for metric in ['close', 'volume', 'divCash', 'splitFactor']:
        # Get data by metric using Tiingo API (requires session
        # config as described at:
        # https://tiingo-python.readthedocs.io/en/latest/readme.html#further-docs)
        wide_frame = client.get_dataframe(symbols,
                                          metric_name = metric,
                                          startDate = start,
                                          endDate = end)

        # Reset index for ease of use and rename (index is untitled
        # if only one symbol is input). Then, melt the data to create
        # observations by date and symbol. Finally, reformat dates.
        wide_frame = wide_frame.reset_index().rename(columns = {'index':'date'})
        metric_df = wide_frame.melt(id_vars = "date").rename(columns = {'variable':'symbol', 'value':metric})
        metric_df['date'] = metric_df['date'].dt.date
        
        # Merge the metric frame to the full frame.
        sample_pricing = sample_pricing.merge(metric_df, on = ['symbol', 'date'], how = 'outer')
            
    return sample_pricing

## S&P 500

Roadmap:
* Read in data.
* Wrangle to get in same format and naming as in strand package.
* Save and check.

In [4]:
sp_wiki = pd.read_csv("sp_wiki.csv")

In [5]:
sp_wiki.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989


In [6]:
# Convert symbol column to list
# Remove problematic symbols due to data procesing issues
sp50 = list(sp_wiki['Symbol'])
issue_list = ['GOOGL', 'BRK.B', 'FRC', 'BF.B', 'OTIS', 
              'CARR', 'CTLT', 'ETSY', 'LUMN', 'TER', 
              'CTL', 'KSS', 'HRB', 'COTY', 'LOW', 
              'UNM', 'AJG']

for ticker in issue_list:
    if ticker in sp50:
        sp50.remove(ticker)

In [7]:
len(sp50)

492

In [8]:
# Need to match lower case in pickle files
low_list = []
for tick in issue_list:
    low_list.append(tick.lower())

## Download fundamental data

#### NOT RUN DUE TO LONG PROCESSING TIMES SKIP AHEAD TO DOWNLOAD FILES

In [18]:
start_time = datetime.now()

sample_inputs_may = get_sample_inputs(sp50, ['2020-05-01'])[['symbol', 'cik', 'total_equity', 'shares_outstanding','end_date']]

end_time = datetime.now()
print('Duration {}'.format(end_time - start_time))

Duration 0:28:47.291655


In [10]:
start_time = datetime.now()

sample_inputs_jun = get_sample_inputs(sp50, ['2020-06-01'])[['symbol', 'cik', 'total_equity', 'shares_outstanding','end_date']]

end_time = datetime.now()
print('Duration {}'.format(end_time - start_time))

Duration 0:17:26.666472


In [11]:
start_time = datetime.now()

sample_inputs_jul = get_sample_inputs(sp50, ['2020-07-01'])[['symbol', 'cik', 'total_equity', 'shares_outstanding','end_date']]

end_time = datetime.now()
print('Duration {}'.format(end_time - start_time))

Duration 0:18:55.627935


In [16]:
start_time = datetime.now()

sample_inputs_aug = get_sample_inputs(sp50, ['2020-08-01'])[['symbol', 'cik', 'total_equity', 'shares_outstanding','end_date']]

end_time = datetime.now()
print('Duration {}'.format(end_time - start_time))

Duration 0:19:56.393579


In [19]:
pd.to_pickle(sample_inputs_may,'sp_sample_inputs_may.pkl')

In [12]:
pd.to_pickle(sample_inputs_jun,'sp_sample_inputs_jun.pkl')

In [15]:
pd.to_pickle(sample_inputs_jul,'sp_sample_inputs_jul.pkl')

In [18]:
pd.to_pickle(sample_inputs_aug,'sp_sample_inputs_aug.pkl')

#### Load fundamental data

In [9]:
sp_inputs_may = pd.read_pickle('sp_sample_inputs_may.pkl')
sp_inputs_jun = pd.read_pickle('sp_sample_inputs_jun.pkl')
sp_inputs_jul = pd.read_pickle('sp_sample_inputs_jul.pkl')
sp_inputs_aug = pd.read_pickle('sp_sample_inputs_aug.pkl')

In [10]:
# Remove problematic tickers from files
sp_inputs_may = sp_inputs_may[~sp_inputs_may['symbol'].isin(low_list)]
sp_inputs_jun = sp_inputs_jun[~sp_inputs_jun['symbol'].isin(low_list)]
sp_inputs_jul = sp_inputs_jul[~sp_inputs_jul['symbol'].isin(low_list)]
sp_inputs_aug = sp_inputs_aug[~sp_inputs_aug['symbol'].isin(low_list)]

In [12]:
print(sp_inputs_may.head())
print(sp_inputs_jun.head())
print(sp_inputs_jul.head())
print(sp_inputs_aug.head())

  symbol      cik  total_equity  shares_outstanding    end_date
0   aapl   320193   89531000000          4375480000  2020-05-01
0   msft   789019  114501000000          7583440247  2020-05-01
0   amzn  1018724   62060000000           497810444  2020-05-01
0     fb  1326801  105304000000          2849103300  2020-05-01
0   goog  1652044  203659000000           682620213  2020-05-01
  symbol      cik  total_equity  shares_outstanding    end_date
0   aapl   320193   78425000000          4334335000  2020-06-01
0   msft   789019  114501000000          7583440247  2020-06-01
0   amzn  1018724   65272000000           498776032  2020-06-01
0     fb  1326801  105304000000          2849103300  2020-06-01
0   goog  1652044  203659000000           682620213  2020-06-01
  symbol      cik  total_equity  shares_outstanding    end_date
0   aapl   320193   78425000000          4334335000  2020-07-01
0   msft   789019  114501000000          7583440247  2020-07-01
0   amzn  1018724   65272000000         

## Download pricing data 

### NOT RUN SKIP TO LOAD PRICING DATA

In [ ]:
start_time = datetime.now()

sp_sample_pricing = get_sample_pricing(sp50, ['2020-05-01', '2020-08-31'])

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [12]:
sp_sample_pricing['date'] = pd.to_datetime(sp_sample_pricing['date'])

In [12]:
pd.to_pickle(sp_sample_pricing, 'sp_sample_pricing.pkl')

#### Load pricing data

In [13]:
sp_sample_pricing = pd.read_pickle('sp_sample_pricing.pkl')

In [14]:
sp_sample_pricing = sp_sample_pricing[~sp_sample_pricing['symbol'].isin(low_list)]

In [15]:
sp_sample_pricing.head()

,date,symbol,close,volume,divCash,splitFactor
0,2020-05-01,aapl,289.07,60154175.0,0.0,1.0
1,2020-05-04,aapl,293.16,33391986.0,0.0,1.0
2,2020-05-05,aapl,297.56,36937795.0,0.0,1.0
3,2020-05-06,aapl,300.63,35583438.0,0.0,1.0
4,2020-05-07,aapl,303.74,28803764.0,0.0,1.0


In [17]:
too_many = []
for sym in sp50:
    if sp_sample_pricing.loc[sp_sample_pricing['symbol'] == sym.lower(), 'symbol'].count() > 85:
        too_many.append(sym)

too_many

[]

If above does not result in empty list use the following code.

In [18]:
duplicates = []
for ticker in too_many:
    dup_sum = sum(sp_sample_pricing.loc[sp_sample_pricing['symbol'] == 'ticker', ['symbol', 'date']].duplicated())
    duplicates.append(dup_sum)
    
duplicates

[]

In [36]:
## Not run
# sp_sample_pricing = sp_sample_pricing.drop_duplicates(subset=['symbol', 'date'])

##### Split into separate months for data wranging

In [19]:
may = (sp_sample_pricing['date'] > '2020-04-30') &  (sp_sample_pricing['date'] <= '2020-05-31')
jun = (sp_sample_pricing['date'] > '2020-05-31') &  (sp_sample_pricing['date'] <= '2020-06-30')
jul = (sp_sample_pricing['date'] > '2020-06-30') &  (sp_sample_pricing['date'] <= '2020-07-31')
aug = (sp_sample_pricing['date'] > '2020-07-31') &  (sp_sample_pricing['date'] <= '2020-08-31')

sp_pricing_may = sp_sample_pricing[may]
sp_pricing_jun = sp_sample_pricing[jun]
sp_pricing_jul = sp_sample_pricing[jul]
sp_pricing_aug = sp_sample_pricing[aug]

## Create monthly data frames

In [20]:
# Function to wrangle data
def wrangle_func(df, start_date):
    
    """ Function to wrangle S&P 500 data from Edgar and Tingo into Strand 'input' data frame."""
    
    # Create inputs df
    input_list = ['symbol', 'dollar_volume', 'book_to_price', 'shares_outstanding', 'mkt_cap_day', 'close','date']
    inputs = df.loc[:,input_list]

    # Create shift list
    shift_list = [x for x in inputs.columns if x not in ['symbol','dollar_volume','date']]

    # Shift values
    inputs.loc[:,shift_list] = inputs.groupby('symbol').apply(lambda x: x[shift_list].shift(1)).values

    # Create rolling 20-day average dollar volume
    inputs.loc[:,'rc_vol'] = inputs.groupby('symbol')['dollar_volume'].transform(lambda x : x.rolling(20).mean()).values

    # Shift rc_vol by one day
    inputs.loc[:,'rc_vol'] = inputs.groupby('symbol')['rc_vol'].shift(1).values

    # Reorder and rename columns to match sample_inputs in package
    x_inputs = inputs.loc[:,['symbol', 'rc_vol', 'book_to_price', 'shares_outstanding', 'mkt_cap_day', 'close', 'date']]
    x_inputs = x_inputs.rename(columns = {'symbol':'id', 'shares_outstanding':'shares_out', 'mkt_cap_day': 'market_cap','close':'price'})

    # Filter to desired start date
    filter_date = x_inputs['date'] >= start_date
    out = x_inputs.loc[filter_date,:]
    
    return out

## June

### June input build

In [21]:
# Create new dataframe to merge inputs and pricing
test_jun = pd.concat([sp_pricing_may, sp_pricing_jun],axis=0) 
test_jun = test_jun.merge(sp_inputs_jun, how='left', on='symbol')

# Add columns
test_jun['mkt_cap_day'] = test_jun['shares_outstanding'] * test_jun['close']
test_jun['book_to_price'] = test_jun['total_equity']/test_jun['mkt_cap_day']
test_jun['dollar_volume'] = test_jun['volume'] * test_jun['close']
# test['date'] = pd.to_datetime(test['date']) # adjust to datetime object for filtering

test_jun.head()

,date,symbol,close,volume,divCash,splitFactor,cik,total_equity,shares_outstanding,end_date,mkt_cap_day,book_to_price,dollar_volume
0,2020-05-01,aapl,289.07,60154175.0,0.0,1.0,320193,78425000000,4334335000,2020-06-01,1.252926e+12,0.062593,1.738877e+10
1,2020-05-04,aapl,293.16,33391986.0,0.0,1.0,320193,78425000000,4334335000,2020-06-01,1.270654e+12,0.061720,9.789195e+09
2,2020-05-05,aapl,297.56,36937795.0,0.0,1.0,320193,78425000000,4334335000,2020-06-01,1.289725e+12,0.060808,1.099121e+10
3,2020-05-06,aapl,300.63,35583438.0,0.0,1.0,320193,78425000000,4334335000,2020-06-01,1.303031e+12,0.060187,1.069745e+10
4,2020-05-07,aapl,303.74,28803764.0,0.0,1.0,320193,78425000000,4334335000,2020-06-01,1.316511e+12,0.059570,8.748855e+09


In [22]:
test_inputs_jun = wrangle_func(test_jun, "2020-06-01")

In [23]:
test_inputs_jun.head()

,id,rc_vol,book_to_price,shares_out,market_cap,price,date
9840,aapl,1.083084e+10,0.056910,4.334335e+09,1.378058e+12,317.94,2020-06-01
9841,aapl,1.028735e+10,0.056218,4.334335e+09,1.395006e+12,321.85,2020-06-02
9842,aapl,1.015212e+10,0.055959,4.334335e+09,1.401464e+12,323.34,2020-06-03
9843,aapl,1.002721e+10,0.055653,4.334335e+09,1.409179e+12,325.12,2020-06-04
9844,aapl,9.845117e+09,0.056136,4.334335e+09,1.397043e+12,322.32,2020-06-05


In [24]:
# Check right number dates
for ticker in sp50:
    if test_inputs_jun[test_inputs_jun['id'] == 'ticker']['date'].nunique() > 24:
        print(ticker)    

## June pricing build

In [25]:
# Create pricing df
pricing = test_jun.loc[:,['symbol', 'close', 'divCash', 'volume','splitFactor', 'date']]

# Shift unadjusted price
# pricing['close_lag'] = pricing['close'].shift(1)
pricing['close_lag'] = pricing.groupby('symbol')['close'].transform(lambda x: x.shift())

# Create distribution_unadj column
pricing['distribution_unadj'] = 0.0

# Create adjustment ratio column
pricing['adjustment_ratio'] = 1/pricing['splitFactor']

# Reorder and rename columns according to sample_pricing in package
x_pricing = pricing.loc[:,['symbol', 'close', 'close_lag','divCash', 'distribution_unadj','volume','adjustment_ratio', 'date']]
x_pricing = x_pricing.rename(columns = {'symbol':'id', 'close':'price_unadj',
                                        'close_lag': 'prior_close_unadj', 'divCash':'dividend_unadj'})
x_pricing.head()

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
0,aapl,289.07,NaN,0.0,0.0,60154175.0,1.0,2020-05-01
1,aapl,293.16,289.07,0.0,0.0,33391986.0,1.0,2020-05-04
2,aapl,297.56,293.16,0.0,0.0,36937795.0,1.0,2020-05-05
3,aapl,300.63,297.56,0.0,0.0,35583438.0,1.0,2020-05-06
4,aapl,303.74,300.63,0.0,0.0,28803764.0,1.0,2020-05-07


Check for odd month end/begin price issues

In [26]:
check = x_pricing.loc[:, ['id','date', 'prior_close_unadj']]
date_check = ['2020-05-29', '2020-06-01']
check = check[check['date'].isin(date_check)].pivot_table(index='id', columns= 'date', values ='prior_close_unadj')
check.columns.name = None

check[abs(check.iloc[:,1]/check.iloc[:,0]-1) > 0.1]

,2020-05-29,2020-06-01
id,,
dxc,16.57,14.21


In [27]:
# Filter to desired start date
test_pricing_jun = x_pricing[x_pricing['date'] >= '2020-06-01']
test_pricing_jun.head()

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
9840,aapl,321.85,317.94,0.0,0.0,20254653.0,1.0,2020-06-01
9841,aapl,323.34,321.85,0.0,0.0,21910704.0,1.0,2020-06-02
9842,aapl,325.12,323.34,0.0,0.0,26122804.0,1.0,2020-06-03
9843,aapl,322.32,325.12,0.0,0.0,21890091.0,1.0,2020-06-04
9844,aapl,331.50,322.32,0.0,0.0,34312550.0,1.0,2020-06-05


### July

#### July input build

In [28]:
# Create new dataframe to merge inputs and pricing
test_jul = pd.concat([sp_pricing_jun, sp_pricing_jul],axis=0) 
test_jul = test_jul.merge(sp_inputs_jul, how='left', on='symbol')

# Add columns
test_jul['mkt_cap_day'] = test_jul['shares_outstanding'] * test_jul['close']
test_jul['book_to_price'] = test_jul['total_equity']/test_jul['mkt_cap_day']
test_jul['dollar_volume'] = test_jul['volume'] * test_jul['close']
# test['date'] = pd.to_datetime(test['date']) # adjust to datetime object for filtering

test_jul.head()

,date,symbol,close,volume,divCash,splitFactor,cik,total_equity,shares_outstanding,end_date,mkt_cap_day,book_to_price,dollar_volume
0,2020-06-01,aapl,321.85,20254653.0,0.0,1.0,320193,78425000000,4334335000,2020-07-01,1.395006e+12,0.056218,6.518960e+09
1,2020-06-02,aapl,323.34,21910704.0,0.0,1.0,320193,78425000000,4334335000,2020-07-01,1.401464e+12,0.055959,7.084607e+09
2,2020-06-03,aapl,325.12,26122804.0,0.0,1.0,320193,78425000000,4334335000,2020-07-01,1.409179e+12,0.055653,8.493046e+09
3,2020-06-04,aapl,322.32,21890091.0,0.0,1.0,320193,78425000000,4334335000,2020-07-01,1.397043e+12,0.056136,7.055614e+09
4,2020-06-05,aapl,331.50,34312550.0,0.0,1.0,320193,78425000000,4334335000,2020-07-01,1.436832e+12,0.054582,1.137461e+10


In [29]:
test_inputs_jul = wrangle_func(test_jul, "2020-07-01")

In [30]:
test_inputs_jul.head()

,id,rc_vol,book_to_price,shares_out,market_cap,price,date
10824,aapl,1.342848e+10,0.049599,4.334335e+09,1.581165e+12,364.80,2020-07-01
10825,aapl,1.350794e+10,0.049693,4.334335e+09,1.578175e+12,364.11,2020-07-02
10826,aapl,1.367975e+10,0.049693,4.334335e+09,1.578175e+12,364.11,2020-07-06
10827,aapl,1.366704e+10,0.048399,4.334335e+09,1.620391e+12,373.85,2020-07-07
10828,aapl,1.379395e+10,0.048549,4.334335e+09,1.615363e+12,372.69,2020-07-08


In [31]:
# Check right number dates
for ticker in sp50:
    if test_inputs_jul[test_inputs_jul['id'] == 'ticker']['date'].nunique() > 24:
        print(ticker)    

### July pricing build

In [32]:
# Create pricing df
pricing = test_jul.loc[:,['symbol', 'close', 'divCash', 'volume','splitFactor', 'date']]

# Shift unadjusted pride
# pricing['close_lag'] = pricing['close'].shift(1)
pricing['close_lag'] = pricing.groupby('symbol')['close'].transform(lambda x: x.shift())

# Create distribution_unadj column
pricing['distribution_unadj'] = 0.0

# Create adjustment ratio column
pricing['adjustment_ratio'] = 1/pricing['splitFactor']

# Reorder and rename columns according to sample_pricing in package
x_pricing = pricing.loc[:,['symbol', 'close', 'close_lag','divCash', 'distribution_unadj','volume','adjustment_ratio', 'date']]
x_pricing = x_pricing.rename(columns = {'symbol':'id', 'close':'price_unadj',
                                        'close_lag': 'prior_close_unadj', 'divCash':'dividend_unadj'})
x_pricing.head()

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
0,aapl,321.85,NaN,0.0,0.0,20254653.0,1.0,2020-06-01
1,aapl,323.34,321.85,0.0,0.0,21910704.0,1.0,2020-06-02
2,aapl,325.12,323.34,0.0,0.0,26122804.0,1.0,2020-06-03
3,aapl,322.32,325.12,0.0,0.0,21890091.0,1.0,2020-06-04
4,aapl,331.50,322.32,0.0,0.0,34312550.0,1.0,2020-06-05


Check for odd month end/begin issues

In [33]:
check = x_pricing.loc[:, ['id','date', 'prior_close_unadj']]
date_check = ['2020-06-30', '2020-07-01']
check = check[check['date'].isin(date_check)].pivot_table(index='id', columns= 'date', values ='prior_close_unadj')
check.columns.name = None
# check[np.where(abs(check.iloc[:,1]/check.iloc[:,0]-1)>0.05)].index

check[abs(check.iloc[:,1]/check.iloc[:,0]-1) > 0.1]

,2020-06-30,2020-07-01
id,,


In [34]:
# Filter to desired start date
test_pricing_jul = x_pricing[x_pricing['date'] >= '2020-07-01']
test_pricing_jul.head()

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
10824,aapl,364.11,364.80,0.0,0.0,27690454.0,1.0,2020-07-01
10825,aapl,364.11,364.11,0.0,0.0,28814516.0,1.0,2020-07-02
10826,aapl,373.85,364.11,0.0,0.0,29745936.0,1.0,2020-07-06
10827,aapl,372.69,373.85,0.0,0.0,28206999.0,1.0,2020-07-07
10828,aapl,381.37,372.69,0.0,0.0,29274479.0,1.0,2020-07-08


## August

### August input build

In [35]:
# Create new dataframe to merge inputs and pricing
test_aug = pd.concat([sp_pricing_jul, sp_pricing_aug],axis=0)
test_aug = test_aug.merge(sp_inputs_aug, how='left', on='symbol')

# Add columns
test_aug['mkt_cap_day'] = test_aug['shares_outstanding'] * test_aug['close']
test_aug['book_to_price'] = test_aug['total_equity']/test_aug['mkt_cap_day']
test_aug['dollar_volume'] = test_aug['volume'] * test_aug['close']
# test['date'] = pd.to_datetime(test['date']) # adjust to datetime object for filtering

test_aug.head()

,date,symbol,close,volume,divCash,splitFactor,cik,total_equity,shares_outstanding,end_date,mkt_cap_day,book_to_price,dollar_volume
0,2020-07-01,aapl,364.11,27690454.0,0.0,1.0,320193,72282000000,4275634000,2020-08-01,1.556801e+12,0.046430,1.008237e+10
1,2020-07-02,aapl,364.11,28814516.0,0.0,1.0,320193,72282000000,4275634000,2020-08-01,1.556801e+12,0.046430,1.049165e+10
2,2020-07-06,aapl,373.85,29745936.0,0.0,1.0,320193,72282000000,4275634000,2020-08-01,1.598446e+12,0.045220,1.112052e+10
3,2020-07-07,aapl,372.69,28206999.0,0.0,1.0,320193,72282000000,4275634000,2020-08-01,1.593486e+12,0.045361,1.051247e+10
4,2020-07-08,aapl,381.37,29274479.0,0.0,1.0,320193,72282000000,4275634000,2020-08-01,1.630599e+12,0.044329,1.116441e+10


In [36]:
test_inputs_aug = wrangle_func(test_aug, "2020-08-01")

In [37]:
test_inputs_aug.head()

,id,rc_vol,book_to_price,shares_out,market_cap,price,date
10824,aapl,1.352567e+10,0.039774,4.275634e+09,1.817315e+12,425.04,2020-08-03
10825,aapl,1.464811e+10,0.038796,4.275634e+09,1.863108e+12,435.75,2020-08-04
10826,aapl,1.506995e+10,0.038539,4.275634e+09,1.875550e+12,438.66,2020-08-05
10827,aapl,1.518307e+10,0.038400,4.275634e+09,1.882348e+12,440.25,2020-08-06
10828,aapl,1.573421e+10,0.037105,4.275634e+09,1.948022e+12,455.61,2020-08-07


In [38]:
# Check right number dates
for ticker in sp50:
    if test_inputs_aug[test_inputs_aug['id'] == 'ticker']['date'].nunique() > 24:
        print(ticker)    

### August pricing build

In [39]:
# Create pricing df
pricing = test_aug.loc[:,['symbol', 'close', 'divCash', 'volume','splitFactor', 'date']]

# Shift unadjusted pride
# pricing['close_lag'] = pricing['close'].shift(1)
pricing['close_lag'] = pricing.groupby('symbol')['close'].transform(lambda x: x.shift())

# Create distribution_unadj column
pricing['distribution_unadj'] = 0.0

# Reorder and rename columns according to sample_pricing in package
# Create adjustment ratio column
pricing['adjustment_ratio'] = 1/pricing['splitFactor']

# Reorder and rename columns according to sample_pricing in package
x_pricing = pricing.loc[:,['symbol', 'close', 'close_lag','divCash', 'distribution_unadj','volume','adjustment_ratio', 'date']]
x_pricing = x_pricing.rename(columns = {'symbol':'id', 'close':'price_unadj',
                                        'close_lag': 'prior_close_unadj', 'divCash':'dividend_unadj'})
x_pricing.head()

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
0,aapl,364.11,NaN,0.0,0.0,27690454.0,1.0,2020-07-01
1,aapl,364.11,364.11,0.0,0.0,28814516.0,1.0,2020-07-02
2,aapl,373.85,364.11,0.0,0.0,29745936.0,1.0,2020-07-06
3,aapl,372.69,373.85,0.0,0.0,28206999.0,1.0,2020-07-07
4,aapl,381.37,372.69,0.0,0.0,29274479.0,1.0,2020-07-08


Check for odd month end/begin issues

In [40]:
check = x_pricing.loc[:, ['id','date', 'prior_close_unadj']]
date_check = ['2020-07-31', '2020-08-03']
check = check[check['date'].isin(date_check)].pivot_table(index='id', columns= 'date', values ='prior_close_unadj')
check.columns.name = None
check[abs(check.iloc[:,1]/check.iloc[:,0]-1) > 0.1]

,2020-07-31,2020-08-03
id,,
aapl,384.76,425.04


In [41]:
# Filter to desired start date
test_pricing_aug = x_pricing[x_pricing['date'] >= '2020-08-01']
test_pricing_aug.head()

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
10824,aapl,435.75,425.04,0.00,0.0,77037847.0,1.0,2020-08-03
10825,aapl,438.66,435.75,0.00,0.0,43198092.0,1.0,2020-08-04
10826,aapl,440.25,438.66,0.00,0.0,30497988.0,1.0,2020-08-05
10827,aapl,455.61,440.25,0.00,0.0,50607225.0,1.0,2020-08-06
10828,aapl,444.45,455.61,0.82,0.0,49511403.0,1.0,2020-08-07


## Combine builds

### Input build

In [42]:
test_inputs = pd.concat([test_inputs_jun, test_inputs_jul, test_inputs_aug], axis = 0)
print(test_inputs.head())
print(test_inputs.tail())

        id        rc_vol  book_to_price    shares_out    market_cap   price  \
9840  aapl  1.083084e+10       0.056910  4.334335e+09  1.378058e+12  317.94   
9841  aapl  1.028735e+10       0.056218  4.334335e+09  1.395006e+12  321.85   
9842  aapl  1.015212e+10       0.055959  4.334335e+09  1.401464e+12  323.34   
9843  aapl  1.002721e+10       0.055653  4.334335e+09  1.409179e+12  325.12   
9844  aapl  9.845117e+09       0.056136  4.334335e+09  1.397043e+12  322.32   

           date  
9840 2020-06-01  
9841 2020-06-02  
9842 2020-06-03  
9843 2020-06-04  
9844 2020-06-05  
        id        rc_vol  book_to_price   shares_out    market_cap  price  \
21151  nws  6.170798e+06       0.896209  588487106.0  8.739034e+09  14.85   
21152  nws  6.215852e+06       0.889619  588487106.0  8.803767e+09  14.96   
21153  nws  6.101736e+06       0.888431  588487106.0  8.815537e+09  14.98   
21154  nws  6.219560e+06       0.886065  588487106.0  8.839076e+09  15.02   
21155  nws  6.201539e+06       0

In [ ]:
# Adjust share-based fundamentals for EW to account for 3:1 split
ew_idx = (test_inputs['id'] == 'ew') & \
    (test_inputs['date'] >= '2020-06-02') & \
    (test_inputs['date'] <= '2020-07-31')

test_inputs[ew_idx, ['book_to_price']] = test_inputs.loc[ew_idx, ['book_to_price']] / 3
test_inputs.loc[ew_idx, ['shares_out']] = test_inputs.loc[ew_idx, ['shares_out']] * 3
test_inputs.loc[ew_idx, ['market_cap']] = test_inputs.loc[ew_idx, ['market_cap']] * 3

### Pricing build

In [43]:
test_pricing = pd.concat([test_pricing_jun, test_pricing_jul, test_pricing_aug], axis=0)
print(test_pricing.head())
print(test_pricing.tail())

        id  price_unadj  prior_close_unadj  dividend_unadj  \
9840  aapl       321.85             317.94             0.0   
9841  aapl       323.34             321.85             0.0   
9842  aapl       325.12             323.34             0.0   
9843  aapl       322.32             325.12             0.0   
9844  aapl       331.50             322.32             0.0   

      distribution_unadj      volume  adjustment_ratio       date  
9840                 0.0  20254653.0               1.0 2020-06-01  
9841                 0.0  21910704.0               1.0 2020-06-02  
9842                 0.0  26122804.0               1.0 2020-06-03  
9843                 0.0  21890091.0               1.0 2020-06-04  
9844                 0.0  34312550.0               1.0 2020-06-05  
        id  price_unadj  prior_close_unadj  dividend_unadj  \
21151  nws        14.96              14.85             0.0   
21152  nws        14.98              14.96             0.0   
21153  nws        15.02          

In [44]:
test_pricing[(test_pricing['adjustment_ratio'] < 1.0)]

,id,price_unadj,prior_close_unadj,dividend_unadj,distribution_unadj,volume,adjustment_ratio,date
12502,ew,74.27,224.72,0.0,0.0,2981101.0,0.333333,2020-06-01
10844,aapl,129.04,499.23,0.0,0.0,223505733.0,0.250000,2020-08-31


## Test data frames for duplicates, odd market cap numbers

In [45]:
sum(test_inputs['id'].value_counts() != 65)

0

In [46]:
sum(test_pricing['id'].value_counts() != 65)

0

In [47]:
test_inputs.loc[test_inputs['market_cap'] > 1e12, 'id'].unique()

array(['aapl', 'msft', 'amzn', 'goog'], dtype=object)

## Secref build

In [48]:
sp_wiki.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989


In [54]:
test_secref = sp_wiki.loc[:, ['Symbol', 'Security', 'GICS Sector']]
test_secref = test_secref[~test_secref['Symbol'].isin(issue_list)]
test_secref = test_secref.rename(columns={'Symbol':'id', 'Security':'name', 'GICS Sector':'sector'})
test_secref.head()

,id,name,sector
0,MMM,3M Company,Industrials
1,ABT,Abbott Laboratories,Health Care
2,ABBV,AbbVie Inc.,Health Care
3,ABMD,ABIOMED Inc,Health Care
4,ACN,Accenture plc,Information Technology


In [55]:
test_secref.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492 entries, 0 to 504
Data columns (total 3 columns):
id        492 non-null object
name      492 non-null object
sector    492 non-null object
dtypes: object(3)
memory usage: 15.4+ KB


### SP build

In [56]:
# Create sp_pricing data frame
# Change symbol to id and end_date to date to have same naming as sp_inputs
# Rearrange column order to match date at end as in samples in strand package  
sp_inputs = test_inputs.copy()
sp_pricing = test_pricing.copy()
sp_secref = test_secref.copy()

### Save data

In [57]:
sp_inputs.to_pickle('sp_inputs.pkl')
sp_inputs.to_csv('sp_inputs.csv', sep=',', index=False)

sp_pricing.to_pickle('sp_pricing.pkl')
sp_pricing.to_csv('sp_pricing.csv', sep=',', index=False)

sp_secref.to_pickle('sp_secref.pkl')
sp_secref.to_csv('sp_secref.csv', sep=',', index=False)

#### Test to ensure data is there

In [58]:
data = ['sp_inputs', 'sp_pricing', 'sp_secref']

for dat in data:
    df = pd.read_pickle("{}.pkl".format(dat))
    print(df.head)
    df = pd.read_csv("{}.csv".format(dat))
    print(df.head)
    
del df

<bound method NDFrame.head of          id        rc_vol  book_to_price    shares_out    market_cap   price  \
9840   aapl  1.083084e+10       0.056910  4.334335e+09  1.378058e+12  317.94   
9841   aapl  1.028735e+10       0.056218  4.334335e+09  1.395006e+12  321.85   
9842   aapl  1.015212e+10       0.055959  4.334335e+09  1.401464e+12  323.34   
9843   aapl  1.002721e+10       0.055653  4.334335e+09  1.409179e+12  325.12   
9844   aapl  9.845117e+09       0.056136  4.334335e+09  1.397043e+12  322.32   
...     ...           ...            ...           ...           ...     ...   
21151   nws  6.170798e+06       0.896209  5.884871e+08  8.739034e+09   14.85   
21152   nws  6.215852e+06       0.889619  5.884871e+08  8.803767e+09   14.96   
21153   nws  6.101736e+06       0.888431  5.884871e+08  8.815537e+09   14.98   
21154   nws  6.219560e+06       0.886065  5.884871e+08  8.839076e+09   15.02   
21155   nws  6.201539e+06       0.873274  5.884871e+08  8.968543e+09   15.24   

         

In [59]:
sp_inputs_jun[sp_inputs_jun['symbol'] == 'ajg']

,symbol,cik,total_equity,shares_outstanding,end_date


In [60]:
test_inputs.loc[test_inputs['market_cap'] > 1e+12, "id"].unique()

array(['aapl', 'msft', 'amzn', 'goog'], dtype=object)